In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertConfig, BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from tqdm import tqdm
from copy import deepcopy
import torch.nn.functional as F
class FeatureHook:
    """Intercepte les activations d'une couche specifique."""
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
        self.features = None

    def hook_fn(self, module, input, output):
        # Pour les transformers, output peut etre un tuple
        if isinstance(output, tuple):
            self.features = output[0]
        else:
            self.features = output

    def close(self):
        self.hook.remove()



class EncResistantMap(nn.Module):
    """Mapping network that projects weights to watermark space."""
    def __init__(self, weight_size, watermark_len, expansion_factor=2):
        super().__init__()
        self.fc1 = nn.Linear(weight_size, 100, bias=True)
        self.fc2 = nn.Linear(100, expansion_factor * weight_size, bias=True)
        self.sig = nn.Sigmoid()
        self.tanh = nn.Tanh()
        # Secret projection matrix
        self.register_buffer(
            "matrix_a",
            torch.randn(weight_size * expansion_factor, watermark_len)
        )

    def forward(self, theta_f):
        out = self.fc1(theta_f)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.sig(out @ self.matrix_a)
        return out

class ResEncryptSentiment:
    def __init__(self, model_id="prajjwal1/bert-tiny", device="cuda"):
        self.device = device
        self.model_id = model_id
        
        # Chargement du modele (bert-tiny: 4.4M params, ~15x plus rapide que DistilBERT)
        # self.model = AutoModelForSequenceClassification.from_pretrained(
        #     model_id, num_labels=2, use_safetensors=True
        # ).to(device)

        config = BertConfig.from_pretrained(model_id, num_labels=2)
        self.model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
        self.tokenizer=BertTokenizer.from_pretrained(model_id)
        # self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        
        # Configuration par defaut
        self.config = {
            # "layer_name": "classifier.weight",  # Couche de classification
             "layer_name": "bert.encoder.layer.1.output.dense.weight",
            "hook_layer":  "bert.encoder.layer.1.output.dense",

            "watermark_len": 64,
            "expansion_factor": 10,
            "lr": 5e-5,  # Learning rate plus eleve pour petit modele
            "lambda_wat": 1.0,
            "lr_mn": 1e-3,  # Learning rate for MappingNet
            "lambda_1": 1.0,   # Watermark loss weight
            "lambda_2": 0.1,   # Original watermark loss weight
            "lambda_3": 0.001,
            "epochs": 30  # Plus d'epochs car modele non-pretrained sur sentiment
        }
        
        self.saved_keys = {}
        
    def list_layers(self):
        """Affiche les couches disponibles pour le watermarking."""
        print("Couches disponibles:")
        for name, param in self.model.named_parameters():
            print(f"  {name}: {param.shape}")

    def _get_target_layer(self, model, layer_name=None):
        """Recupere la couche cible pour le hook."""

        for name, module in model.named_modules():
            if name == layer_name:
                return module
        raise ValueError(f"Couche {layer_name} introuvable.")



    def _get_target_weights(self, model):
        """Recupere le tenseur des poids de la couche cible."""
        for name, param in model.named_parameters():
            if name == self.config["layer_name"]:
                return param
        raise ValueError(f"Parametre {self.config['layer_name']} introuvable.")

    def _get_mean(self, weights):
        """Compute mean based on layer type."""
        if len(weights.shape) == 4:  # Conv layer
            return weights.mean(dim=(0, 1))
        elif len(weights.shape) == 2:  # Linear layer
            return weights.mean(dim=0)
        else:
            raise NotImplementedError("Layer type not supported")


    def embed(self, train_dataloader, eval_dataloader=None):
        """
        Incorpore la marque Res_Encrypt pendant le finetuning.
        """
        print(f"--- Demarrage Embedding Res_Encrypt ({self.config['layer_name']}) ---")



        # 1. Preparation des modeles
        original_model = deepcopy(self.model)
        original_model.eval()
        for p in original_model.parameters():
            p.requires_grad = False


        watermarked_model = self.model
        watermarked_model.train()
        
        # 2. Generation des Cles
        target_weights = self._get_target_weights(watermarked_model)
        with torch.no_grad():
            theta_f = torch.flatten(self._get_mean(target_weights))
            weight_size = len(theta_f)


        
        print(f"Dimension vecteur poids : {weight_size} | Watermark : {self.config['watermark_len']} bits")
        
        # Watermarks
        watermark_target = torch.randint(0, 2, (self.config["watermark_len"],)).float().to(self.device)
        watermark_random = torch.randint(0, 2, (self.config["watermark_len"],)).float().to(self.device)



        # MappingNet
        mapping_net = EncResistantMap(
            weight_size,
            self.config["watermark_len"],
            self.config["expansion_factor"]
        ).to(self.device)



        # Original weights (frozen)
        orig_weights = self._get_target_weights(original_model)
        theta_fn = torch.flatten(self._get_mean(orig_weights)).detach()

        # 3. Optimiseur
        optimizer = torch.optim.AdamW([
            {'params': watermarked_model.parameters(), 'lr': self.config["lr"]},
            {'params': mapping_net.parameters(), 'lr': self.config["lr_mn"]}
        ])




        ce_loss = nn.CrossEntropyLoss()
        bce_loss = nn.BCELoss()
        
        # 3. Boucle d'entrainement
        for epoch in range(self.config["epochs"]):
            pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{self.config['epochs']}")
            total_loss = 0
            correct = 0
            total = 0
            
            for batch in pbar:
                input_ids = batch["input_ids"].to(self.device)  #tokennized textes (numbers)
                attention_mask = batch["attention_mask"].to(self.device) #which tokens to pay attention to (1 for real tokens, 0 for padding)
                labels = batch["labels"].to(self.device) #correct answer for each sentence ( 0 = negative, 1 = positive)
                
                optimizer.zero_grad()
                
                # A. Forward pass (Task Loss)
                outputs = watermarked_model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                l_main = ce_loss(logits, labels)
                
                # B. Watermark Loss (Regularization)
                current_weights = self._get_target_weights(watermarked_model)
                theta_f = torch.flatten(self._get_mean(current_weights))

                # Projection via MappingNet
                matrix_g = mapping_net(theta_f.unsqueeze(0)).squeeze(0)
                matrix_gn = mapping_net(theta_fn.unsqueeze(0)).squeeze(0)

                # Loss: watermarked model -> target watermark
                l_wat = bce_loss(matrix_g, watermark_target)
                # Loss: original model -> random watermark (discriminative)
                l_wat_orig = bce_loss(matrix_gn, watermark_random)
                l_l1_w = torch.norm(theta_f, p=1)

                # Loss Totale
                l_total = (l_main +
                          self.config["lambda_1"] * l_wat +
                          self.config["lambda_2"] * l_wat_orig +
                          self.config["lambda_3"] * l_l1_w)
                
                l_total.backward()
                optimizer.step()
                
                # Metrics
                total_loss += l_main.item()
                preds = torch.argmax(logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
                
                ber = self._compute_ber(matrix_g, watermark_target)
                acc = 100. * correct / total
                pbar.set_postfix(L_Main=f"{l_main.item():.3f}", L_Wat=f"{l_wat.item():.3f}", 
                                BER=f"{ber:.2f}", Acc=f"{acc:.1f}%")
            
            # Evaluation en fin d'epoch
            if eval_dataloader:
                eval_acc = self.evaluate(watermarked_model, eval_dataloader)
                print(f"Epoch {epoch+1} | Eval Accuracy: {eval_acc:.2f}%")
        
        # Sauvegarde des cles
        self.saved_keys = {
             "watermark_target": watermark_target,
            "watermarked_model": watermarked_model,
            "mapping_net": mapping_net,
            "original_unet": original_model,
        }
        
        final_ber = self._compute_ber(matrix_g, watermark_target)
        print(f"\n--- Embedding termine | BER final: {final_ber:.2f} ---")
        return watermarked_model
    
    def evaluate(self, model, dataloader):
        """Evalue la precision du modele."""
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        model.train()
        return 100. * correct / total
    
    def extract(self, suspect_model=None):
        """
        Extrait la marque d'un modele suspect.
        """
        if suspect_model is None:
            suspect_model = self.saved_keys["watermarked_model"]
        
        mapping_net = self.saved_keys["mapping_net"]
        watermark_target = self.saved_keys["watermark_target"]

        mapping_net.eval()
        
        # 1. Recuperation des poids
        try:
            target_weights = self._get_target_weights(suspect_model)
        except ValueError:
            print("Couche cible introuvable dans le modele suspect.")
            return 1.0, None
        
        # 2. Projection
        with torch.no_grad():
            theta_f = torch.flatten(self._get_mean(target_weights))
            pred_wm_prob = mapping_net(theta_f.unsqueeze(0)).squeeze(0)
            ber = self._compute_ber(pred_wm_prob, watermark_target)
        
        print(f"BER Extrait : {ber:.2f}")
        return ber, pred_wm_prob
    
    @staticmethod
    def _compute_ber(pred, target):
        return ((pred > 0.5).float() != target).float().mean().item()

print("Classe res_encryptSentiment chargee (bert-tiny: 4.4M params)")

/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Classe res_encryptSentiment chargee (bert-tiny: 4.4M params)


In [2]:
# --- Chargement des donnees SST-2 ---

# Charger le dataset SST-2 (sentiment binaire)
dataset = load_dataset("glue", "sst2")

# Initialiser le watermarker (charge le modele bert-tiny)
res_encrypt = ResEncryptSentiment(device="cuda" if torch.cuda.is_available() else "cpu")

# Afficher les couches disponibles
res_encrypt.list_layers()

# Tokenization
def tokenize_function(examples):
    return res_encrypt.tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True) #converts raw text sentences into input_ids and attention_mask
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# DataLoaders (batch_size=64 possible grace au petit modele)
train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=64, shuffle=True)
eval_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=64)

print(f"\nTrain samples: {len(tokenized_dataset['train'])}")
print(f"Eval samples: {len(tokenized_dataset['validation'])}")

Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 1890.72it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.we

Couches disponibles:
  bert.embeddings.word_embeddings.weight: torch.Size([30522, 128])
  bert.embeddings.position_embeddings.weight: torch.Size([512, 128])
  bert.embeddings.token_type_embeddings.weight: torch.Size([2, 128])
  bert.embeddings.LayerNorm.weight: torch.Size([128])
  bert.embeddings.LayerNorm.bias: torch.Size([128])
  bert.encoder.layer.0.attention.self.query.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.self.query.bias: torch.Size([128])
  bert.encoder.layer.0.attention.self.key.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.self.key.bias: torch.Size([128])
  bert.encoder.layer.0.attention.self.value.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.self.value.bias: torch.Size([128])
  bert.encoder.layer.0.attention.output.dense.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.output.dense.bias: torch.Size([128])
  bert.encoder.layer.0.attention.output.LayerNorm.weight: torch.Size([128])
  bert.encoder.layer

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1821/1821 [00:00<00:00, 34834.41 examples/s]


Train samples: 67349
Eval samples: 872


In [3]:

# Lancer l'embedding
watermarked_model = res_encrypt.embed(train_dataloader, eval_dataloader)

--- Demarrage Embedding Res_Encrypt (bert.encoder.layer.1.output.dense.weight) ---
Dimension vecteur poids : 512 | Watermark : 64 bits


Epoch 1/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:05<00:00, 204.17it/s, Acc=79.8%, BER=0.00, L_Main=0.264, L_Wat=0.000]


Epoch 1 | Eval Accuracy: 80.96%


Epoch 2/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:05<00:00, 206.91it/s, Acc=88.5%, BER=0.00, L_Main=0.345, L_Wat=0.000]


Epoch 2 | Eval Accuracy: 82.68%


...

Epoch 28 | Eval Accuracy: 78.78%


Epoch 29/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:04<00:00, 217.33it/s, Acc=98.7%, BER=0.00, L_Main=0.160, L_Wat=0.000]


Epoch 29 | Eval Accuracy: 78.56%


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:04<00:00, 217.85it/s, Acc=98.7%, BER=0.00, L_Main=0.006, L_Wat=0.000]


Epoch 30 | Eval Accuracy: 78.21%

--- Embedding termine | BER final: 0.00 ---


In [4]:
# --- Test d'Extraction ---

print("=" * 50)
print("TEST D'EXTRACTION")
print("=" * 50)

# Extraction sur le modele watermarke
print("\n1. Extraction sur le modele watermarke:")
ber_watermarked, _ = res_encrypt.extract(watermarked_model)

# Extraction sur un modele vierge (non-watermarke)
print("\n2. Extraction sur un modele vierge (devrait avoir BER ~0.5):")

model_id= "prajjwal1/bert-tiny"
config = BertConfig.from_pretrained(model_id, num_labels=2)
fresh_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device="cuda" if torch.cuda.is_available() else "cpu")
tokenizer=BertTokenizer.from_pretrained(model_id)

ber_fresh, _ = res_encrypt.extract(fresh_model)

print(f"\nResume:")
print(f"  - BER modele watermarke: {ber_watermarked:.2f} (attendu: ~0.0)")
print(f"  - BER modele vierge: {ber_fresh:.2f} (attendu: ~0.5)")

TEST D'EXTRACTION

1. Extraction sur le modele watermarke:
BER Extrait : 0.00

2. Extraction sur un modele vierge (devrait avoir BER ~0.5):


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 2256.68it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.we

BER Extrait : 0.52

Resume:
  - BER modele watermarke: 0.00 (attendu: ~0.0)
  - BER modele vierge: 0.52 (attendu: ~0.5)


In [7]:
# --- Fonction de Distillation (Attaque) ---

def run_distillation_attack(res_encrypt_obj, train_dataloader, eval_dataloader, epochs=5, lr=5e-5):
    """
    Tente de transferer la fonctionnalite du modele watermarke vers un modele vierge.
    Verifie si la marque (basee sur les poids) survit.
    """
    device = res_encrypt_obj.device
    
    # 1. Teacher (Gele)
    teacher_model = res_encrypt_obj.saved_keys["watermarked_model"]
    teacher_model.eval()
    for p in teacher_model.parameters():
        p.requires_grad = False
    
    # 2. Student (Vierge - Meme architecture)
    config = BertConfig.from_pretrained(model_id, num_labels=2)
    print("\n--- Initialisation du Student ---")
    student_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
    student_model.train()
    
    # Verification initiale
    teacher_ber, _ = res_encrypt_obj.extract(teacher_model)
    student_ber, _ = res_encrypt_obj.extract(student_model)
    print(f"[Check] BER Teacher: {teacher_ber:.2f}")
    print(f"[Check] BER Student (Avant): {student_ber:.2f}")
    
    optimizer = torch.optim.AdamW(student_model.parameters(), lr=lr)
    history = {"loss": [], "ber": [], "acc": []}
    
    print(f"\n--- Distillation ({epochs} epochs) ---")
    feature_criterion = nn.MSELoss()
    for epoch in range(epochs):
        pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
        running_loss = 0.0
        
        for batch in pbar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            
            optimizer.zero_grad()

            teacher_layer = res_encrypt._get_target_layer(teacher_model, res_encrypt.config["hook_layer"])
            hook = FeatureHook(teacher_layer)
            
            # Output Matching (Soft Labels)
            with torch.no_grad():
                teacher_logits = teacher_model(input_ids=input_ids, attention_mask=attention_mask).logits
                teacher_probs = F.softmax(teacher_logits / 2.0, dim=-1)  # Temperature=2
                t_features = hook.features
            hook.close()

            student_layer = res_encrypt._get_target_layer(student_model, res_encrypt.config["hook_layer"])
            hook = FeatureHook(student_layer)
            student_logits = student_model(input_ids=input_ids, attention_mask=attention_mask).logits
            student_log_probs = F.log_softmax(student_logits / 2.0, dim=-1)
            s_features = hook.features
            hook.close()


            loss_feat = feature_criterion(s_features.mean(dim=[0, 1]), t_features.mean(dim=[0, 1])) + feature_criterion(s_features.var(dim=[0, 1]), t_features.var(dim=[0, 1]))


            # KL Divergence Loss
            loss_kl = F.kl_div(student_log_probs, teacher_probs, reduction="batchmean") * (2.0 ** 2)
            loss= loss_kl +loss_feat
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            pbar.set_postfix(Loss=f"{loss.item():.4f}")
        
        # Evaluation
        avg_loss = running_loss / len(train_dataloader)
        history["loss"].append(avg_loss)
        
        current_ber, pred_wm_prob = res_encrypt_obj.extract(student_model)
        history["ber"].append(current_ber)
        
        eval_acc = res_encrypt_obj.evaluate(student_model, eval_dataloader)
        history["acc"].append(eval_acc)
        
        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | BER: {current_ber:.2f} | Acc: {eval_acc:.1f}% err_wat: {nn.BCELoss()(pred_wm_prob, res_encrypt_obj.saved_keys["watermark_target"]).item()}")
    
    return student_model, history

In [8]:
# --- Lancer l'attaque par distillation ---

student_model, attack_history = run_distillation_attack(
    res_encrypt,
    train_dataloader, 
    eval_dataloader,
    epochs= 1000, # Plus d'epochs possible car modele rapide
)


--- Initialisation du Student ---


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 2871.55it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.we

BER Extrait : 0.00
BER Extrait : 0.52
[Check] BER Teacher: 0.00
[Check] BER Student (Avant): 0.52

--- Distillation (1000 epochs) ---


Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:05<00:00, 185.87it/s, Loss=0.3713]


BER Extrait : 0.52
Epoch 1 | Loss: 1.4737 | BER: 0.52 | Acc: 81.4% err_wat: 22.661540985107422


...

BER Extrait : 0.00
Epoch 730 | Loss: 0.0053 | BER: 0.00 | Acc: 79.4% err_wat: 0.15162800252437592


Epoch 731: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:06<00:00, 166.04it/s, Loss=0.0044]


BER Extrait : 0.00
Epoch 731 | Loss: 0.0059 | BER: 0.00 | Acc: 79.2% err_wat: 0.145213782787323


Epoch 732:  44%|███████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                                       | 466/1053 [00:02<00:03, 173.23it/s, Loss=0.0038]


KeyboardInterrupt: 

In [ ]:
# --- Visualisation des Resultats ---

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
axes[0].plot(attack_history["loss"], 'b-o')
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Distillation Loss")
axes[0].set_title("Loss durant Distillation")
axes[0].grid(True)

# BER
axes[1].plot(attack_history["ber"], 'r-o')
axes[1].axhline(y=0.5, color='gray', linestyle='--', label='Random (0.5)')
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("BER")
axes[1].set_title("BER du Student (Watermark)")
axes[1].set_ylim([0, 1])
axes[1].legend()
axes[1].grid(True)

# Accuracy
axes[2].plot(attack_history["acc"], 'g-o')
axes[2].set_xlabel("Epoch")
axes[2].set_ylabel("Accuracy (%)")
axes[2].set_title("Accuracy du Student")
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Resume final
print("\n" + "=" * 50)
print("RESUME FINAL")
print("=" * 50)
print(f"BER Teacher (watermarke): {res_encrypt.extract(res_encrypt.saved_keys['watermarked_model'])[0]:.2f}")
print(f"BER Student (apres distillation): {attack_history['ber'][-1]:.2f}")
print(f"Accuracy Student: {attack_history['acc'][-1]:.1f}%")

if attack_history['ber'][-1] > 0.4:
    print("\n-> La marque res_encrypt n'a PAS survecu a la distillation (BER eleve)")
else:
    print("\n-> La marque res_encrypt a survecu a la distillation (BER faible)")